# libtorch integration test

This notebook validates that `libtorch.py` integrates SymPy expressions using Torch + torchquad.

It is intentionally minimal and should run on CPU-only setups; CUDA is optional.

In [2]:

from __future__ import annotations

import sys
from pathlib import Path

import torch


def _find_repo_root_with_src(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "src" / "libtorch.py").exists():
            return p
    raise FileNotFoundError("Could not find 'src/libtorch.py' above current working directory")


ROOT = _find_repo_root_with_src(Path.cwd().resolve())
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print("cwd:", Path.cwd())
print("src:", SRC)
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("cuda:", torch.version.cuda)

cwd: c:\Users\Pc\Desktop\LOCALS\F_Nutku_Studies\libphysics\tests
src: C:\Users\Pc\Desktop\LOCALS\F_Nutku_Studies\libphysics\src
torch: 2.5.1+cu121
cuda available: True
cuda: 12.1


In [3]:
import math

import sympy as sp

from libtorch import torchify, torchquad_integrate, setup_logging

torch.set_default_dtype(torch.float64)


print("sympy:", sp.__version__)

sympy: 1.13.1


### Integral expressions for both tests

Test 1 (complex Gaussian, over (-∞, ∞))  
Let x, k ∈ ℝ.  
$$
\int_{-\infty}^{\infty} e^{-x^{2}} e^{i k x}\,dx = \sqrt{\pi}\, e^{-k^{2}/4}.
$$

Test 2 (semi-infinite exponential, over (0, ∞))  
Let x ∈ ℝ.  
$$
\int_{0}^{\infty} e^{-x}\,dx = 1.
$$

In [8]:
# Test 1: complex Gaussian integral over (-oo, oo)
# ∫ exp(-x^2) * exp(I*k*x) dx = sqrt(pi) * exp(-k^2/4)

x, k = sp.symbols("x k", real=True)
expr = sp.exp(-x**2) * sp.exp(sp.I * k * x)

texpr = torchify(
    expr,
    variables=[x],
    limits=[(x, -sp.oo, sp.oo)],
    params=[k],
)

re, im = torchquad_integrate(texpr, params_values=[1.0], N=151)

expected = math.sqrt(math.pi) * math.exp(-(1.0**2) / 4.0)
re_f = float(re.detach().cpu())
im_f = float(im.detach().cpu())

print("re:", re_f, "expected:", expected, "abs err:", abs(re_f - expected))
print("im:", im_f)

assert abs(re_f - expected) < 5e-3
assert abs(im_f) < 5e-3

re: 1.380388447018694 expected: 1.380388447043143 abs err: 2.4448887359085347e-11
im: 7.806255641895632e-18


In [9]:
# Test 2: semi-infinite integral over (0, oo)
# ∫_0^∞ exp(-x) dx = 1

x = sp.Symbol("x", real=True)
expr = sp.exp(-x)

texpr = torchify(
    expr,
    variables=[x],
    limits=[(x, 0, sp.oo)],
    params=[],
)

re, im = torchquad_integrate(texpr, params_values=[], N=151)
re_f = float(re.detach().cpu())
im_f = float(im.detach().cpu())

print("re:", re_f, "expected:", 1.0, "abs err:", abs(re_f - 1.0))
print("im:", im_f)

assert abs(re_f - 1.0) < 5e-3
assert abs(im_f) < 5e-3

re: 0.9999999997326827 expected: 1.0 abs err: 2.6731727942319594e-10
im: 0.0
